*Image credit: The Labour Party*

*See the full impact estimate for 2025 [here](https://policyengine.org/uk/policy?reform=68829&focus=policyOutput.policyBreakdown&region=uk&timePeriod=2025&baseline=1).*


## Key findings



1. PolicyEngine estimates that four reported tax reforms to Income Tax, National Insurance, Capital Gains Tax and VAT would raise £23 billion in FY2025.
2. Our model estimates that these reforms would be progressive, reducing income inequality by 0.6%, and have no significant impact on the government’s absolute before-housing-costs poverty rate.


## Background

On 30th October, the Chancellor Rachel Reeves will announce proposals for the UK’s tax and spending decisions (and any changes to them) over the next five years in the first Autumn Budget of the new Labour government. [Media](https://www.bbc.co.uk/news/articles/cdxl1zd07l1o) [reports](https://www.theguardian.com/uk-news/2024/oct/17/budget-2024-uk-tax-raises-rachel-reeves) expect these decisions to include both existing manifesto commitments (VAT on private school fees and non-dom tax reform) as well as reforms not included in Labour’s last election manifesto (employer National Insurance and Capital Gains Tax reform).

In this report, we outline PolicyEngine’s estimates of the household sector impacts of each of these four most reported-on policy reforms, and their combined impact if all are announced together.


## Policy choices


### Tax avoidance

Labour says [in their manifesto](https://labour.org.uk/change/labours-fiscal-plan/) they would ‘close non-dom tax loopholes’ and ‘close the carried interest tax loophole’ as well as invest in reducing tax avoidance. They said this would raise on average £5.9 billion each year over the parliament in their manifesto. The impact of this reform on households is highly uncertain: as an approximation, we represent this as a tax on earnings over £100,000.


### Private school VAT

In their manifesto, Labour also committed to levying VAT (as well as business rates) on private school fees from FY2025. We estimate this raises around £1.5 billion per year, assuming no behavioural responses, mostly from higher-income deciles. Read more detail about the policy in our recently-published report [here](https://policyengine.org/uk/research/school-vat).


### Employer NICs

The UK government is [reportedly](https://www.bbc.co.uk/news/articles/cly4leklk3qo) considering raising National Insurance Contributions (NICs) for employers, by levying NICs on employers' contributions to workers’ pensions. Currently employers pay 13.8% tax on earnings above around £9,500. We use a behavioural model that assumes 40% employee wage incidence, 30% consumer incidence and 30% capital incidence in the first year, rising to 70% employee wage incidence and 15% consumer-capital incidence by the fifth.


### Capital Gains Tax

Robert Peston of ITV [reports](https://x.com/Peston/status/1847000929913983451) that the Chancellor will raise the higher and additional rate of capital gains from 20% to 24%, raising ‘a few billion’. 

We impute capital gains to households in our microdata using administrative data published in CAGE Working Paper no. 465, Capital Gains and UK Inequality (Advani &Arun Advani, Andy Summers, 2020) ([here](https://warwick.ac.uk/fac/soc/economics/research/centres/cage/manage/publications/wp465.2020.pdf)). We assume an elasticity of capital gains with respect to the marginal tax rate on capital gains income of -0.5.[^1]


## Economic impacts

PolicyEngine [estimates](https://policyengine.org/uk/policy?reform=68829&focus=policyOutput.policyBreakdown&region=uk&timePeriod=2025&baseline=1) that these reforms combined would raise £25 billion per year over five years, with over half of the revenue impact from the employer NICs reform. The table below summarises these reforms’ impacts over the budget window (with the behavioural responses as stated previously).


| Year | 2025 | 2026 | 2027 | 2028 | 2029 | 2025-29 |
|------|------|------|------|------|------|---------|
| Levy employer pension NICs | 14.1 | 13.8 | 13.2 | 12.6 | 12.9 | 66.6 |
| Levy private school VAT | 1.5 | 1.5 | 1.5 | 1.6 | 1.6 | 7.7 |
| Raise CGT to 24% | 1.6 | 1.9 | 2.0 | 2.3 | 2.3 | 10.1 |
| Reduce tax avoidance | 5.9 | 5.9 | 5.9 | 5.9 | 5.9 | 29.5 |
| **Total** | **[23.1](https://policyengine.org/uk/policy?reform=69049&focus=policyOutput.distributionalImpact.incomeDecile.relative&region=uk&timePeriod=2025&baseline=1)** | **23.1** | **22.6** | **22.4** | **22.7** | **113.9** |




[^1]: We will publish an in-depth report detailing our methodology (see our open-source implementation here) in future.

In [2]:
from policyengine_uk import Microsimulation
from policyengine_core.reforms import Reform

REDUCE_TAX_AVOIDANCE = {
    "gov.contrib.policyengine.budget.high_income_incident_tax_change": {
        "2025-01-01.2030-12-31": 5.9,
    },
}

PRIVATE_SCHOOL_VAT = {
    "gov.contrib.labour.private_school_vat": {
        "2025-01-01.2030-12-31": 0.2,
    }
}

EMPLOYER_NICS = {
    "gov.contrib.policyengine.employer_ni.exempt_employer_pension_contributions": {
        "2025-01-01.2030-12-31": False,
    },
    "gov.contrib.policyengine.employer_ni.employee_incidence": {
        "2025-01-01.2025-12-31": 0.4,
        "2026-01-01.2026-12-31": 0.5,
        "2027-01-01.2027-12-31": 0.6,
        "2028-01-01.2028-12-31": 0.7,
        "2029-01-01.2030-12-31": 0.7,
    },
    "gov.contrib.policyengine.employer_ni.consumer_incidence": {
        "2025-01-01.2030-12-31": 0.5,
    },
    "gov.contrib.policyengine.employer_ni.capital_incidence": {
        "2025-01-01.2030-12-31": 0.5,
    },
}

CGT = {
    "gov.hmrc.cgt.higher_rate": {
        "2025-01-01.2030-12-31": 0.24,
    },
    "gov.hmrc.cgt.additional_rate": {
        "2025-01-01.2030-12-31": 0.24,
    },
    "gov.simulation.capital_gains_responses.elasticity": {
        "2025-01-01.2030-12-31": -0.5,
    }
}

baseline = Microsimulation()

import pandas as pd

decile = baseline.calculate("household_income_decile", 2025).clip(1, 10)

reform_labels = [
    "Reduce tax avoidance",
    "Levy private school VAT",
    "Levy employer pension NICs",
    "Raise CGT to 24%",
]

reforms = [
    REDUCE_TAX_AVOIDANCE,
    PRIVATE_SCHOOL_VAT,
    EMPLOYER_NICS,
    CGT,
]

df_reform = []
df_decile = []
df_value = []

from tqdm import tqdm

baseline_deciles = (
    baseline.calculate("gov_balance", 2025).groupby(decile).sum()
)
last_deciles = None

for i in range(len(reforms)):
    combined_reform = {}
    for j in range(i + 1):
        combined_reform.update(reforms[j])

    reform = Reform.from_dict(combined_reform)

    reformed = Microsimulation(reform=reform)

    deciles = (
        reformed.calculate("gov_balance", 2025).groupby(decile).sum()
    )

    if last_deciles is None:
        last_deciles = baseline_deciles

    decile_change = deciles - last_deciles

    last_deciles = deciles

    df_reform.extend([reform_labels[i]] * 10)
    df_decile.extend(list(range(1, 11)))
    df_value.extend(decile_change.values)

df = pd.DataFrame(
    {
        "Reform": df_reform,
        "Decile": df_decile,
        "Impact": df_value,
    }
)

import plotly.express as px
from policyengine_core.charts import format_fig

fig = px.bar(
    df,
    x="Decile",
    y="Impact",
    color="Reform",
    color_discrete_sequence=px.colors.qualitative.T10,
).update_layout(
    xaxis_title="Income decile",
    yaxis_title="Change in revenues (£ billion)",
    title="Revenue impact of possible reforms by household income decile",
    showlegend=True,
    legend_title="Reform",
    xaxis_tickvals=list(range(1, 11)),
)

# Add a label to each bar combination with the total revenue

for i in range(1, 11):
    fig.add_annotation(
        x=i,
        y=df[df["Decile"] == i]["Impact"].sum(),
        text=f"{df[df['Decile'] == i]['Impact'].sum()/1e9:,.1f}",
        showarrow=False,
        # offset down a bit
        yshift=15,
    )

format_fig(fig)

The distributional impact is progressive, and we estimate that these reforms would reduce the Gini coefficient of income inequality by 0.6%. Just under half (47%) of the revenue impact comes from the richest income decile (mostly from the reforms other than employer NICs). 

The Employer NICs reform is mostly flat (as a percentage of income) across the income distribution in FY2025, because of the lower employee incidence. Assuming a higher consumption/capital incidence for this reform creates a more regressive (with respect to income) impact, while assuming a higher employee incidence moves the impact to be progressive along income deciles. The 40/30/30 split in the first year approximately balances these out. The Times [reports](https://www.thetimes.com/uk/politics/article/rachel-reeves-to-protect-public-sector-workers-from-tax-raid-0n25hxkgg) that the Treasury estimates the employer NICs reform would raise £15.4 billion in FY2025 (our estimate is 7% lower).

Absolute, before-housing-costs poverty as measured by the government’s Households Below Average Income definition does not change, because none of the reforms except for the wage incidence of employer NICs affect the definition of income used for this metric.


## Conclusion

PolicyEngine estimates that these four reforms would raise £23 billion per year over the five-year budget window. We also estimate that these would be progressive in that they would reduce the Gini coefficient of income inequality, but that they would not change the government’s central poverty rates on their own.


## Appendix: behavioural responses

For our estimates of the impacts of reforms to Capital Gains Tax (CGT) and employer National Insurance, we assume that individuals with capital gains respond to changes in their marginal tax rate and employers respond to changes in their tax liability by adjusting employee wages. We do not account for behavioural responses to private school VAT or reduced tax avoidance measures.

The table below shows the impact of these behavioural responses on the total five-year revenue impact of each provision, and in total. Overall, we estimate that behavioural responses to these tax reforms would lower the revenue impact by around 25%.


| Reform | Static revenue impact | Dynamic revenue impact | Absolute change | Relative change |
|--------|---------------------|----------------------|-----------------|-----------------|
| Levy employer pension NICs | 90.3 | 66.6 | -23.7 | -26% |
| Levy private school VAT | 7.7 | 7.7 | 0 | 0% |
| Raise CGT to 24% | 21.5 | 10.1 | -11.4 | -53% |
| Reduce tax avoidance | 5.9 | 5.9 | 0 | 0 |
| **Total** | **149.0** | **113.9** | **-35.1** | **-24%** |